<a href="https://colab.research.google.com/github/WeiChuen99/Social-Feedback-Enhanced-Argument-Claim-Extraction-from-News-Headlines/blob/main/Social_Feedback_Enhanced_Argument_Claim_Extraction_from_News_Headlines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [ ]:
%%capture
# !pip3 install flair

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 300)
# import praw #reddit data api
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sn
import re #regex

# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer #VADER sentiment model
import requests
import json
import csv
import time
import datetime
import tensorflow as tf
from tensorflow import keras
tf.test.gpu_device_name() #run to make sure tensorflow is connected to gpu (if applicable)

import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.translate.bleu_score import sentence_bleu 
#import flair

from textblob import TextBlob
import os

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [ ]:
%%capture
pip install stop-words

In [ ]:
%%capture
 !pip install pyLDAvis==3.2.2

In [ ]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.wordnet import WordNetLemmatizer
from gensim import corpora, models
import pandas as pd
import gensim
import pyLDAvis.gensim
from nltk.corpus import stopwords 
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

import pprint
import warnings

from pandas import read_excel

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


#Reddit Scraping

In [ ]:
#reddit submissions scrapping

def get_pushshift_data(query, after, before, sub):
  # url = 'https://api.pushshift.io/reddit/search/submission/?title=' + str(query) + '&size=500&after=' + str(
  #     after) + '&before=' + str(before) + '&subreddit=' + str(sub)
  url = 'https://api.pushshift.io/reddit/search/submission/?after=' + str(
      after) + '&before=' + str(before) + '&subreddit=' + str(sub)
  print(url)
  r = requests.get(url)
  try:
    data = json.loads(r.text)
  except:
    print("server busy")
    return []
  return data['data']


def collect_sub_data(subm):
  sub_data = list()  # list to store data points
  headline = subm['title']
  sub_id = subm['id']
  created = datetime.datetime.fromtimestamp(subm['created_utc'])
  numComms = subm['num_comments']

  #only use submissions with sufficient comments
  if numComms > 100:
    sub_data.append((sub_id, headline, created, numComms))
    sub_stats[sub_id] = sub_data


def write_subs_to_file(filename):
  upload_count = 0
  if os.path.exists(filename):
      keep_header = False
  else:
      keep_header = True

  with open(filename, 'a', newline='') as file:
      a = csv.writer(file, delimiter=',')
      headers = ['sub_id', 'headline', 'publish_date', 'num_of_comments']
      if keep_header:
          a.writerow(headers)

      for sub in all_subs:
          a.writerow(all_subs[sub][0])
          upload_count += 1


if __name__ == '__main__':
  # Download reddit posts from sub_reddit with keywords given by key_word

  sub_reddit = 'science'
  key_word = 'covid'

  output_filename = 'reddit_data.csv'
  # search all the posts from start_date to end_date overall
  start_date = datetime.datetime(2020, 11, 11, 0)
  end_date = datetime.datetime(2020, 11, 15, 0)

  # in each itration get reddit posts for one day, to avoid getting blocked by server
  one_day = datetime.timedelta(hours=24)
  after_date = start_date
  after = str(int(after_date.timestamp()))
  before_date = start_date + one_day
  before = str(int(before_date.timestamp()))

  all_subs = {}
  while after_date < end_date:
    print('-' * 80)
    print(after_date, ' -> ', before_date)
    print('-' * 80)

    sub_count = 0
    sub_stats = {}
    sub_ids = []

    data = get_pushshift_data(key_word, after, before, sub_reddit)

    max_count = 100
    count = 0
    while len(data) > 0 and count < max_count:
        # print('count ', count)
        for submission in data:
            collect_sub_data(submission)
            sub_count += 1

        # print(len(data))
        print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
        after = data[-1]['created_utc']
        data = get_pushshift_data(key_word, after, before, sub_reddit)
        all_subs.update(sub_stats)
        # print(data)
        # print(data['data'][0]['author'])
        count = count + 1

    # move to next day
    after_date += one_day
    after = str(int(after_date.timestamp()))
    before_date += one_day
    before = str(int(before_date.timestamp()))

    # randomly sleep before starting next iteration
    time.sleep(np.random.randint(1, 3))

  # keep saving data collected in each iteration
  write_subs_to_file(output_filename)

In [1]:
sheet_name = 'reddit_data' # change it to your sheet name, you can find your sheet name at the bottom left of your excel file
file_name = 'reddit_data_labeled.xlsx' # change it to the name of your excel file
df_labeled = read_excel(file_name, sheet_name = sheet_name)

NameError: ignored

In [ ]:
#read csv into dataframe
# df_subs = pd.read_csv('reddit_data.csv', usecols= ['post_id', 'headline', 'publish_date', 'num_of_comments'])
df_subs = df_labeled

In [ ]:
#reddit comments scrapping

def get_pushshift_com_data(post_id, limit):
    #https://api.pushshift.io/reddit/comment/search/?link_id=j32xmk&limit=2&sort=asc
    # url = 'https://api.pushshift.io/reddit/comment/search/?link_id=' + str(post_id) + '&limit=' + str(
    #     limit) + '&sort=asc'
    url = 'https://api.pushshift.io/reddit/comment/search/?link_id=' + str(post_id) + '&limit=' + str(
        limit)+ '&sort=asc'
    print(url)
    r = requests.get(url)
    try:
      url_data = json.loads(r.text)
    except:
      print("server error")
      return []
    return url_data['data']


def collect_com_data(subm):
  com_id = subm['id']
  parent_sub_id = subm['link_id']  
  parent_sub_id = parent_sub_id[3:] #id of submission
  parent_id = subm['parent_id']     #to determine level of comment
  body = subm['body']               #comment body
  score = subm['score']             #upvotes or downvotes

  return com_id, parent_sub_id, parent_id, body, score

def write_coms_to_file(filename):
    upload_count = 0
    if os.path.exists(filename):
        keep_header = False
    else:
        keep_header = True

    with open(filename, 'a', newline='') as file:
        a = csv.writer(file, delimiter=',')
        headers = ['sub_id', 'comment']
        if keep_header:
            a.writerow(headers)
        for com in com_stats:
            a.writerow(com_stats[com][0])
            upload_count += 1

if __name__ == '__main__':
  com_stats = {}
  com_output_filename = "reddit_com_data.csv"
  limit_num_comments = 500
  ####### will increase later if using first level comments only 

  for sub_id in df_subs["sub_id"]:
    data = []  # list to store data points
    merged_body = ""
    num_comments = 0
    com_data = get_pushshift_com_data(sub_id, limit_num_comments)
    for comment in com_data:
      com_id, parent_sub_id, parent_id, body, score = collect_com_data(comment)
      if body != "[deleted]" and body != "[removed]" and body != "" and parent_id[0:2] == "t3" and len(body) > 6:
        merged_body = merged_body + body
        num_comments = num_comments + 1
        if num_comments == 10:
          break
        
    data.append((parent_sub_id, merged_body))
    com_stats[sub_id] = data

    #randomly sleep before starting next iteration
    time.sleep(np.random.randint(1, 3))

  # keep saving data collected in each iteration
  write_coms_to_file(com_output_filename)

In [ ]:
#read csv into dataframe
df_coms = pd.read_csv('reddit_com_data.csv', usecols= ['sub_id', 'comment'])

In [ ]:
import json

# Function to convert a CSV to JSON
# Takes the file paths as arguments
def make_json(csvFilePath, jsonFilePath):
    # Open a csv reader called DictReader
    with open(csvFilePath, encoding='utf-8') as csvf:
      csvReader = csv.DictReader(csvf)

      # Convert each row into a dictionary 
      # and add it to data
      for rows in csvReader:
      # Open a json writer, and use the json.dumps() 
      # function to dump data
        with open(jsonFilePath, 'a', encoding='utf-8') as jsonf:
            jsonf.write(json.dumps(rows))
            jsonf.write("\n")
         
# Driver Code
 
# Decide the two file paths according to your 
# computer system
subcsvFilePath = r'reddit_data.csv'
subjsonFilePath = r'reddit_data.json'

comcsvFilePath = r'reddit_com_data.csv'
comjsonFilePath = r'reddit_com_data.json'

# Call the make_json function
# make_json(subcsvFilePath, subjsonFilePath)
# make_json(comcsvFilePath, comjsonFilePath)


#Topic Modeling using LDA

In [ ]:
#read csv into dataframe
df_labeled = pd.read_csv('reddit_data_evaluated.csv', usecols= ['headlines', 'comment', 'labeled_argument'])

In [ ]:
df_labeled

In [ ]:
#ignore depreciated warnings
warnings.filterwarnings('ignore')

def preprocess_data (document):
  #Initiating Tokenizer, Lemmatizer and Stop Words
  pattern = r'\b[^\d\W]+\b'
  tokenizer = RegexpTokenizer(pattern)
  en_stop = get_stop_words('en')
  lemmatizer = WordNetLemmatizer()
  remove_words = list(stopwords.words('english'))

  #Perform Tokenization, Words removal, and Lemmatization

  # clean and tokenize document string
  raw = str(document).lower()
  tokens = tokenizer.tokenize(raw)

  # remove stop words from tokens
  stopped_tokens = [raw for raw in tokens if not raw in en_stop]
  
  # remove stop words from tokens
  stopped_tokens_new = [raw for raw in stopped_tokens if not raw in remove_words]
  
  # lemmatize tokens
  lemma_tokens = [lemmatizer.lemmatize(tokens) for tokens in stopped_tokens_new]
  
  # remove word containing only single char
  new_lemma_tokens = [raw for raw in lemma_tokens if not len(raw) == 1]
  
  # add tokens to list
  texts = []
  texts.append(new_lemma_tokens)

  # turn our tokenized documents into a id <-> term dictionary
  dictionary = corpora.Dictionary(texts)
  # convert tokenized documents into a document-term matrix
  corpus = [dictionary.doc2bow(text) for text in texts]
  
  return dictionary, corpus

def perform_lda (document):
  # list for tokenized documents in loop
  lda_topics = []
  lda_models = []
  list_corpus = []
  list_dictionary = []

  for row in document:
    #preprocess data
    dictionary, corpus = preprocess_data(row)

    #Generate LDA model
    ldamodel = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=1, id2word = dictionary, passes=20)
    topics = ldamodel.top_topics(corpus,topn=7)

    #convert topic array into string
    topics = topics[0][0]
    topics_str = ""
    for topic_word in topics:
      topics_str = topics_str + " " + topic_word[1]


    lda_models.append(ldamodel)
    lda_topics.append(topics_str)
    list_corpus.append(corpus)
    list_dictionary.append(dictionary)

  return lda_topics, lda_models, list_corpus, list_dictionary


baseline_lda, baseline_models, baseline_corpus, baseline_dictionary = perform_lda(df_labeled["headlines"])
df_labeled["headline_lda"] = baseline_lda

df_labeled["headline_and_comment"] = df_labeled["headlines"] + df_labeled["comment"]
proposed_lda, proposed_models, proposed_corpus, proposed_dictionary = perform_lda(df_labeled["headline_and_comment"])
df_labeled["social_aspect_lda"] = proposed_lda

In [ ]:
#create new dataframe with relevent data
dfnew = pd.concat([df_labeled["headlines"], df_labeled["labeled_argument"],
                   df_labeled["headline_lda"], df_labeled["social_aspect_lda"]], axis = 1)
dfnew.columns = ["headlines", "labeled_argument", "headline_lda", "social_aspect_lda"]

In [ ]:
def evaluate(dataframe, list_string):
  list_score = []
  for index, row in dataframe.iterrows():
    reference = row[list_string[0]]
    candidate = row[list_string[1]]
    score = sentence_bleu([reference.split(" ")], candidate.split(" "))
    list_score.append(score)
  print("average bleu score = ", sum(list_score)/len(list_score))
  return list_score

#evaluate baseline method
baseline_bleu_score = evaluate(dfnew, ["labeled_argument", "headline_lda"])

#evaluate social aspect method
social_aspect_bleu_score = evaluate(dfnew, ["labeled_argument", "social_aspect_lda"])

In [ ]:
dfnew["baseline_bleu_score"] = baseline_bleu_score
dfnew["social_aspect_bleu_score"] = social_aspect_bleu_score

In [ ]:
dfnew.to_excel(r'reddit_data_evaluated.xlsx', sheet_name='reddit_data', index = False)

In [ ]:
#Visualise LDA model
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_models[0], list_corpus[0], list_dictionary[0], mds='mmds')
vis

#Sentiment Analysis

In [ ]:
#sentiment analysis

# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer #VADER sentiment model

#using flair pre-trained model
flair_sentiment = flair.models.TextClassifier.load('en-sentiment')
fmt = '%Y-%m-%d %H:00:00'
#vader sentiment model
sid = SentimentIntensityAnalyzer()


def get_sentiment_val_for_flair(sentiments):
    """
    parse input of the format [NEGATIVE (0.9284018874168396)] and return +ve or -ve float value
    :param sentiments:
    :return:
    """
    total_sentiment = str(sentiments)
    neg = 'NEGATIVE' in total_sentiment
    if neg:
        total_sentiment = total_sentiment.replace('NEGATIVE', '')
    else:
        total_sentiment = total_sentiment.replace('POSITIVE', '')

    total_sentiment = total_sentiment.replace('(', '').replace('[', '').replace(')', '').replace(']', '')

    val = float(total_sentiment)
    if neg:
        return -val
    return val


def get_sentiment_report(input_filename, output_filename):
  #read csv input
    df = pd.read_csv(input_filename)
    df = df[['title', 'selftext', 'publish_date']]
    df = df.fillna('')

    df['text'] = df['title'] + ' ' + df['selftext']
    df.set_index('publish_date', inplace=True)
    df.drop(['title', 'selftext'], axis=1, inplace=True)

    for row_i, row in df.iterrows():
        tb_sentiment_polarity_dict = dict()
        tb_sentiment_subjectivity_dict = dict()
        flair_sentiment_dict = dict()

        sid_pos_dict = dict()
        sid_neg_dict = dict()
        sid_neu_dict = dict()
        sid_com_dict = dict()

        data = row['text']
        print(row_i)
        print(data[0:15])
        flair_s = flair.data.Sentence(data)
        flair_sentiment.predict(flair_s)
        flair_total_sentiment = flair_s.labels
        flair_val = get_sentiment_val_for_flair(flair_total_sentiment)

        flair_sentiment_dict[str(row_i)] = flair_val
        tb_sentiment_polarity_dict[str(row_i)] = TextBlob(data).sentiment[0]
        tb_sentiment_subjectivity_dict[str(row_i)] = TextBlob(data).sentiment[1]

        ss = sid.polarity_scores(data)
        sid_pos_dict[str(row_i)] = ss['pos']
        sid_neg_dict[str(row_i)] = ss['neg']
        sid_neu_dict[str(row_i)] = ss['neu']
        sid_com_dict[str(row_i)] = ss['compound']

        flair_df = pd.DataFrame.from_dict(flair_sentiment_dict, orient='index', columns=['reddit_flair'])
        flair_df.index.name = 'timestamp'

        tb_polarity_df = pd.DataFrame.from_dict(tb_sentiment_polarity_dict, orient='index',
                                                columns=['reddit_tb_polarity'])
        tb_polarity_df.index.name = 'timestamp'

        tb_subjectivity_df = pd.DataFrame.from_dict(tb_sentiment_subjectivity_dict, orient='index',
                                                    columns=['reddit_tb_subjectivity'])
        tb_subjectivity_df.index.name = 'timestamp'

        sid_pos_df = pd.DataFrame.from_dict(sid_pos_dict, orient='index',
                                            columns=['reddit_sid_pos'])
        sid_pos_df.index.name = 'timestamp'

        sid_neg_df = pd.DataFrame.from_dict(sid_neg_dict, orient='index',
                                            columns=['reddit_sid_neg'])
        sid_neg_df.index.name = 'timestamp'

        sid_neu_df = pd.DataFrame.from_dict(sid_neu_dict, orient='index',
                                            columns=['reddit_sid_neu'])
        sid_neu_df.index.name = 'timestamp'

        sid_com_df = pd.DataFrame.from_dict(sid_com_dict, orient='index',
                                            columns=['reddit_sid_com'])
        sid_com_df.index.name = 'timestamp'

        final_senti_df = pd.concat([flair_df, tb_polarity_df, tb_subjectivity_df, sid_pos_df, sid_neg_df, sid_neu_df,
        							sid_com_df], axis=1)

        if os.path.exists(output_filename):
            keep_header = False
        else:
            keep_header = True

        final_senti_df.to_csv(output_filename, mode='a', header=keep_header)

    return


def clean_sentiment_report(input_filename, output_filename):
    # drop duplicates and sort
    master_df = pd.read_csv(input_filename, index_col=0)
    master_df.index = pd.to_datetime(master_df.index)
    idx = np.unique(master_df.index, return_index=True)[1]
    master_df = master_df.iloc[idx]
    master_df.to_csv(output_filename)


def bucketize_sentiment_report(input_filename, output_filename):
    start_date_time_obj = datetime.datetime(2020, 10, 1, 0)
    end_date_time_obj = datetime.datetime(2020, 10, 31, 0)
    hr1 = datetime.timedelta(hours=1)
    curr_date_time_obj = start_date_time_obj
    in_df = pd.read_csv(input_filename)


    out_dict = dict()

    while curr_date_time_obj <= end_date_time_obj:
        curr_timestamp = curr_date_time_obj.strftime(format=fmt)
        # print(curr_timestamp)
        # create data dict with all possible timestamps and dummy value of reddit_flair
        # reddit_flair is chosen just randomly as a placeholder
        out_dict[curr_timestamp] = 0
        curr_date_time_obj += hr1

    out_df = pd.DataFrame.from_dict(out_dict, orient='index',
                                    columns=['reddit_flair'])

    # print(out_dict)
    out_df.index.name = 'timestamp'
    # populate more colums
    out_df['reddit_flair_count'] = 0
    out_df['reddit_tb_polarity'] = 0
    out_df['reddit_tb_polarity_count'] = 0
    out_df['reddit_tb_subjectivity'] = 0
    out_df['reddit_tb_subjectivity_count'] = 0
    out_df['reddit_sid_neg'] = 0
    out_df['reddit_sid_neu'] = 0
    out_df['reddit_sid_com'] = 0
    out_df['reddit_sid_count'] = 0

    for i in range(len(in_df)):
        timestamp = in_df.loc[i, 'timestamp']
        out_key = datetime.datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S')
        # timestamp is current plus few minutes or seconds, so collect all these data in the bucket of next hour
        out_key += hr1
        out_key = out_key.strftime(format='%Y-%m-%d %H:00:00')
        #print(out_key)
        # add up all values and count how many values we have added. In next pass we would normalize the values
        try:
            out_df.loc[out_key, 'reddit_flair'] += in_df.loc[i, 'reddit_flair']
            out_df.loc[out_key, 'reddit_flair_count'] += 1
            out_df.loc[out_key, 'reddit_tb_polarity'] += in_df.loc[i, 'reddit_tb_polarity']
            out_df.loc[out_key, 'reddit_tb_polarity_count'] += 1
            out_df.loc[out_key, 'reddit_tb_subjectivity'] += in_df.loc[i, 'reddit_tb_subjectivity']
            out_df.loc[out_key, 'reddit_tb_subjectivity_count'] += 1
            out_df.loc[out_key, 'reddit_sid_pos'] += in_df.loc[i, 'reddit_sid_pos']
            out_df.loc[out_key, 'reddit_sid_neg'] += in_df.loc[i, 'reddit_sid_neg']
            out_df.loc[out_key, 'reddit_sid_neu'] += in_df.loc[i, 'reddit_sid_neu']
            out_df.loc[out_key, 'reddit_sid_com'] += in_df.loc[i, 'reddit_sid_com']
            out_df.loc[out_key, 'reddit_sid_count'] += 1
        except:
            pass

    # make timestamp as a column and reindex the dataframe to make loc method happy
    out_df['timestamp'] = out_df.index
    out_df.index = range(len(out_df))

    for i in range(len(out_df)):
        #print(out_df.loc[i, 'timestamp'])
        # normalize the values
        if out_df.loc[i, 'reddit_flair_count'] == 0:
            out_df.loc[i, 'reddit_flair'] = 0
        else:
            out_df.loc[i, 'reddit_flair'] /= out_df.loc[i, 'reddit_flair_count']

        if out_df.loc[i, 'reddit_tb_polarity_count'] == 0:
            out_df.loc[i, 'reddit_tb_polarity'] = 0
        else:
            out_df.loc[i, 'reddit_tb_polarity'] /= out_df.loc[i, 'reddit_tb_polarity_count']

        if out_df.loc[i, 'reddit_tb_subjectivity_count'] == 0:
            out_df.loc[i, 'reddit_tb_subjectivity'] = 0
        else:
            out_df.loc[i, 'reddit_tb_subjectivity'] /= out_df.loc[i, 'reddit_tb_subjectivity_count']

        if out_df.loc[i, 'reddit_sid_count'] == 0:
            out_df.loc[i, 'reddit_sid_pos'] = 0
            out_df.loc[i, 'reddit_sid_neg'] = 0
            out_df.loc[i, 'reddit_sid_neu'] = 0
            out_df.loc[i, 'reddit_sid_com'] = 0
        else:
            out_df.loc[i, 'reddit_sid_pos'] /= out_df.loc[i, 'reddit_sid_count']
            out_df.loc[i, 'reddit_sid_neg'] /= out_df.loc[i, 'reddit_sid_count']
            out_df.loc[i, 'reddit_sid_neu'] /= out_df.loc[i, 'reddit_sid_count']
            out_df.loc[i, 'reddit_sid_com'] /= out_df.loc[i, 'reddit_sid_count']

        if os.path.exists(output_filename):
            keep_header = False
        else:
            keep_header = True

    out_df.drop(['reddit_flair_count', 'reddit_tb_polarity_count', 'reddit_tb_subjectivity_count','reddit_sid_count'], axis=1,
                inplace=True)
    # change back index to timestamp to save the data in csv
    out_df.set_index('timestamp', inplace=True)
    out_df.to_csv(output_filename)


if __name__ == '__main__':
    input_filename = 'reddit_data.csv'
    output_sentiment_filename = input_filename[0:-4] + '_sentiment.csv'

    # read input_filename (which can be generated by download_data_from_reddit.py script) and performs
    # sentiment analyis of the text data
    get_sentiment_report(input_filename, output_sentiment_filename)
    output_sentiment_bucketize_filename = output_sentiment_filename[0:-4] + '_bucketized.csv'

    # reddit posts can land anytime. Collect all the posts (and its sentiment reports) landed on a given hour (0 to 59 minutes)
    # and bucketize them all into the corresponding hour
    bucketize_sentiment_report(output_sentiment_filename, output_sentiment_bucketize_filename)

# Aspect Based Argument Mining (testing)(dataset only)

In [ ]:
#!/usr/bin/env python

import pandas as pd

#################################################################################
# SENTENCES

ABAM_DATA_SENTENCES_without_sentences = pd.read_csv(
        '/content/drive/MyDrive/Colab Notebooks/FYP/ABAM_DATA_SENTENCES.tsv', 
        sep='\t')
print(len(ABAM_DATA_SENTENCES_without_sentences))

ABAM_SENTENCES = pd.read_csv(
        '/content/drive/MyDrive/Colab Notebooks/FYP/ABAM_SENTENCES.tsv', 
        sep='\t')
print(len(ABAM_SENTENCES))

assert len(ABAM_DATA_SENTENCES_without_sentences)==len(ABAM_SENTENCES)

ABAM_DATA_SENTENCES = pd.merge(
        ABAM_DATA_SENTENCES_without_sentences[['topic', 'sentence_hash', 'stance', 'aspect', 'inner', 'cross']], 
        ABAM_SENTENCES, 
        on='sentence_hash')
ABAM_DATA_SENTENCES = ABAM_DATA_SENTENCES[['topic', 'sentence_hash', 'sentence', 'stance', 'aspect', 'inner', 'cross']]
print(len(ABAM_DATA_SENTENCES))

print(ABAM_DATA_SENTENCES[['topic', 'sentence_hash']].groupby('topic').count())

#################################################################################
# SEGMENTS

ABAM_DATA_SEGMENTS_without_segments = pd.read_csv(
        '/content/drive/MyDrive/Colab Notebooks/FYP/ABAM_DATA_SEGMENTS.tsv', 
        sep='\t')
print(len(ABAM_DATA_SEGMENTS_without_segments))

ABAM_SEGMENTS = pd.read_csv(
        '/content/drive/MyDrive/Colab Notebooks/FYP/ABAM_SEGMENTS.tsv', 
        sep='\t')


print(len(ABAM_SEGMENTS))

assert len(ABAM_DATA_SEGMENTS_without_segments)==len(ABAM_SEGMENTS)

ABAM_DATA_SEGMENTS = pd.merge(
        ABAM_DATA_SEGMENTS_without_segments[['topic', 'sentence_hash', 'segment_count', 'segment_hash', 'stance', 'aspect', 'inner', 'cross']], 
        ABAM_SEGMENTS, on='segment_hash')
ABAM_DATA_SEGMENTS = ABAM_DATA_SEGMENTS[['topic', 'sentence_hash', 'segment_count', 'segment_hash', 'segment', 'stance', 'aspect', 'inner', 'cross']]
print(len(ABAM_DATA_SEGMENTS))

print(ABAM_DATA_SEGMENTS[['topic', 'segment_hash']].groupby('topic').count())

#Topic Modelling Ori code 

In [ ]:
#Initiating Tokenizer, Lemmatizer and Stop Words
pattern = r'\b[^\d\W]+\b'
tokenizer = RegexpTokenizer(pattern)
en_stop = get_stop_words('en')
lemmatizer = WordNetLemmatizer()
remove_words = list(stopwords.words('english'))

#Perform Tokenization, Words removal, and Lemmatization
df["Description"] = df["headline"]+". " +df["short_description"]
# list for tokenized documents in loop
texts = []

# loop through document list
for i in df['Description'].iteritems():
    # clean and tokenize document string
    raw = str(i[1]).lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [raw for raw in tokens if not raw in en_stop]
    
    # remove stop words from tokens
    stopped_tokens_new = [raw for raw in stopped_tokens if not raw in remove_words]
    
    # lemmatize tokens
    lemma_tokens = [lemmatizer.lemmatize(tokens) for tokens in stopped_tokens_new]
    
    # remove word containing only single char
    new_lemma_tokens = [raw for raw in lemma_tokens if not len(raw) == 1]
    
    # add tokens to list
    texts.append(new_lemma_tokens)

# sample data
print(texts[0])

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

#Generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=15, id2word = dictionary, passes=20)
import pprint
pprint.pprint(ldamodel.top_topics(corpus,topn=5))

pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

# Standardise installed frameworks version